# Notebook 01b – 4-Hour Intraday Features for Daily SPY

**Goal:** Build daily intraday summary features from 4-hour SPY candles and merge them into the existing daily dataset.

This notebook will:

1. Download 4-hour SPY OHLCV data.
2. Compute per-4h returns and basic derived values.
3. Aggregate 4h data into daily intraday summaries (counts, fractions, volatility, range, etc.).
4. Merge these intraday features with `daily_merged.parquet`.
5. Save the new dataset as `daily_with_4h.parquet` for use in later models.


In [30]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import yfinance as yf

ROOT = Path("..").resolve()
DATA_RAW = ROOT / "data" / "raw"
DATA_PROC = ROOT / "data" / "processed"

DATA_RAW.mkdir(parents=True, exist_ok=True)
DATA_PROC.mkdir(parents=True, exist_ok=True)

END_DATE = pd.Timestamp.today().strftime("%Y-%m-%d")
START_DATE = (pd.Timestamp.today() - pd.Timedelta(days=720)).strftime("%Y-%m-%d")

ROOT, DATA_RAW, DATA_PROC


(WindowsPath('C:/Users/KDP only/Documents/ANN_Final_Project/spy-ann'),
 WindowsPath('C:/Users/KDP only/Documents/ANN_Final_Project/spy-ann/data/raw'),
 WindowsPath('C:/Users/KDP only/Documents/ANN_Final_Project/spy-ann/data/processed'))

In [31]:
daily_path = DATA_PROC / "daily_merged.parquet"
df_daily = pd.read_parquet(daily_path)
df_daily["date"] = pd.to_datetime(df_daily["date"]).dt.date

print("=== DAILY DATA PREVIEW ===")
display(df_daily.head())
print("Shape:", df_daily.shape)
print("Date range:", df_daily["date"].min(), "→", df_daily["date"].max())


=== DAILY DATA PREVIEW ===


,date,Close,High,Low,Open,Volume,ret_1d,log_ret_1d,ma_close_5,ma_close_20,vol_5,vol_20,future_price,future_ret_1d,label_up
0,2010-02-02,83.059364,83.217386,81.930636,82.216584,216327900,0.012104,0.012031,82.055548,84.347997,0.012653,0.010585,82.645493,-0.004983,0
1,2010-02-03,82.645493,83.134609,82.404697,82.683113,172730700,-0.004983,-0.004995,82.055548,84.205024,0.012873,0.010574,80.094604,-0.030865,0
2,2010-02-04,80.094604,82.043540,80.079552,82.005919,356715700,-0.030865,-0.031352,81.734995,83.931498,0.018783,0.012403,80.260124,0.002067,1
3,2010-02-05,80.260124,80.425666,78.694953,80.184871,493585800,0.002067,0.002064,81.625131,83.648186,0.018457,0.012344,79.680710,-0.007219,0
4,2010-02-08,79.680710,80.764291,79.620510,80.320322,224166900,-0.007219,-0.007245,81.148059,83.321606,0.015917,0.012270,80.681526,0.012560,1


Shape: (3753, 15)
Date range: 2010-02-02 → 2024-12-30


In [32]:
spy_4h_path = DATA_RAW / "spy_4h.csv"

if spy_4h_path.exists():
    print("Loading existing 4h SPY data...")
    df_4h = pd.read_csv(spy_4h_path, parse_dates=["datetime"])
else:
    print("Downloading 4h SPY candles from Yahoo...")
    df_4h = yf.download("SPY", start=START_DATE, end=END_DATE, interval="4h")
    df_4h = df_4h.rename_axis("datetime").reset_index()
    df_4h.to_csv(spy_4h_path, index=False)

print("=== RAW 4H DATA PREVIEW ===")
display(df_4h.head())
print("Shape:", df_4h.shape)
print("Date range:", df_4h["datetime"].min(), "→", df_4h["datetime"].max())
print(df_4h.dtypes)


Loading existing 4h SPY data...
=== RAW 4H DATA PREVIEW ===


,datetime,Close,High,Low,Open,Volume
0,NaT,SPY,SPY,SPY,SPY,SPY
1,2023-12-18 14:30:00+00:00,472.5249938964844,472.6499938964844,470.79998779296875,470.9800109863281,29780492
2,2023-12-18 18:30:00+00:00,472.010009765625,472.9800109863281,471.95001220703125,472.5299987792969,24167610
3,2023-12-19 14:30:00+00:00,473.9700012207031,474.4750061035156,472.45001220703125,472.5299987792969,28651433
4,2023-12-19 18:30:00+00:00,474.82000732421875,474.9200134277344,473.7701110839844,473.9800109863281,18659784


Shape: (984, 6)
Date range: 2023-12-18 14:30:00+00:00 → 2025-12-05 18:30:00+00:00
datetime    datetime64[ns, UTC]
Close                    object
High                     object
Low                      object
Open                     object
Volume                   object
dtype: object


In [33]:
df_4h["date"] = df_4h["datetime"].dt.date

# Use Adj Close if available, else Close
price_col_4h = "Adj Close" if "Adj Close" in df_4h.columns else "Close"

numeric_cols = ["Open", "High", "Low", "Close", "Volume"]
df_4h[numeric_cols] = df_4h[numeric_cols].apply(pd.to_numeric, errors="coerce")

df_4h["ret_4h"] = df_4h[price_col_4h].pct_change()
df_4h["is_up"] = (df_4h[price_col_4h] > df_4h["Open"]).astype("int64")

print("=== 4H WITH RETURNS PREVIEW ===")
display(df_4h.head())


=== 4H WITH RETURNS PREVIEW ===


,datetime,Close,High,Low,Open,Volume,date,ret_4h,is_up
0,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,0
1,2023-12-18 14:30:00+00:00,472.524994,472.649994,470.799988,470.980011,29780492.0,2023-12-18,NaN,1
2,2023-12-18 18:30:00+00:00,472.010010,472.980011,471.950012,472.529999,24167610.0,2023-12-18,-0.001090,0
3,2023-12-19 14:30:00+00:00,473.970001,474.475006,472.450012,472.529999,28651433.0,2023-12-19,0.004152,1
4,2023-12-19 18:30:00+00:00,474.820007,474.920013,473.770111,473.980011,18659784.0,2023-12-19,0.001793,1


In [34]:
agg_funcs = {
    "ret_4h": ["mean", "std"],
    "is_up": ["sum", "count"],
    "High": ["max"],
    "Low": ["min"],
}

df_4h_grouped = df_4h.groupby("date").agg(agg_funcs)
df_4h_grouped.columns = [
    "_".join(col).strip() for col in df_4h_grouped.columns.to_flat_index()
]
df_4h_grouped = df_4h_grouped.reset_index()

# Rename for clarity
df_4h_grouped = df_4h_grouped.rename(columns={
    "ret_4h_mean": "intraday_mean_ret_4h",
    "ret_4h_std": "intraday_std_ret_4h",
    "is_up_sum": "intraday_n_up_4h",
    "is_up_count": "intraday_n_candles_4h",
    "High_max": "intraday_high_max",
    "Low_min": "intraday_low_min",
})

# Additional derived features
df_4h_grouped["intraday_frac_up_4h"] = (
    df_4h_grouped["intraday_n_up_4h"] / df_4h_grouped["intraday_n_candles_4h"]
)

df_4h_grouped["intraday_range_4h"] = (
    df_4h_grouped["intraday_high_max"] - df_4h_grouped["intraday_low_min"]
)

# Last 4h return of each day
last_4h = df_4h.sort_values("datetime").groupby("date").tail(1)[["date", "ret_4h"]]
last_4h = last_4h.rename(columns={"ret_4h": "intraday_last_ret_4h"})

df_intraday = df_4h_grouped.merge(last_4h, on="date", how="left")

print("=== DAILY INTRADAY FEATURES PREVIEW ===")
display(df_intraday.head())
print("Shape:", df_intraday.shape)


=== DAILY INTRADAY FEATURES PREVIEW ===


,date,intraday_mean_ret_4h,intraday_std_ret_4h,intraday_n_up_4h,intraday_n_candles_4h,intraday_high_max,intraday_low_min,intraday_frac_up_4h,intraday_range_4h,intraday_last_ret_4h
0,2023-12-18,-0.001090,NaN,1,2,472.980011,470.799988,0.5,2.180023,-0.001090
1,2023-12-19,0.002973,0.001668,2,2,474.920013,472.450012,1.0,2.470001,0.001793
2,2023-12-20,-0.007190,0.012402,1,2,475.894989,467.820007,0.5,8.074982,-0.015960
3,2023-12-21,0.005020,0.000708,1,2,472.975006,468.839996,0.5,4.135010,0.005520
4,2023-12-22,0.000988,0.004317,1,2,475.380005,471.700012,0.5,3.679993,-0.002065


Shape: (494, 10)


In [37]:
df_daily_merged = df_daily.merge(df_intraday, on="date", how="left")

print("=== DAILY + 4H FEATURES PREVIEW ===")
display(df_daily_merged.head())
print("Shape:", df_daily_merged.shape)

# Check for missing intraday days (e.g., holidays)
missing_intraday = df_daily_merged["intraday_n_candles_4h"].isna().sum()
print("Rows missing intraday data:", missing_intraday)

df_daily_merged = df_daily_merged.dropna(subset=["intraday_n_candles_4h"]).reset_index(drop=True)

print("After dropping days without intraday data:", df_daily_merged.shape)
print("=== DAILY + 4H FEATURES PREVIEW ===")
display(df_daily_merged.head())


=== DAILY + 4H FEATURES PREVIEW ===


,date,Close,High,Low,Open,Volume,ret_1d,log_ret_1d,ma_close_5,ma_close_20,...,label_up,intraday_mean_ret_4h,intraday_std_ret_4h,intraday_n_up_4h,intraday_n_candles_4h,intraday_high_max,intraday_low_min,intraday_frac_up_4h,intraday_range_4h,intraday_last_ret_4h
0,2010-02-02,83.059364,83.217386,81.930636,82.216584,216327900,0.012104,0.012031,82.055548,84.347997,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-02-03,82.645493,83.134609,82.404697,82.683113,172730700,-0.004983,-0.004995,82.055548,84.205024,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-02-04,80.094604,82.043540,80.079552,82.005919,356715700,-0.030865,-0.031352,81.734995,83.931498,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-02-05,80.260124,80.425666,78.694953,80.184871,493585800,0.002067,0.002064,81.625131,83.648186,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-02-08,79.680710,80.764291,79.620510,80.320322,224166900,-0.007219,-0.007245,81.148059,83.321606,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Shape: (3753, 24)
Rows missing intraday data: 3493
After dropping days without intraday data: (260, 24)
=== DAILY + 4H FEATURES PREVIEW ===


,date,Close,High,Low,Open,Volume,ret_1d,log_ret_1d,ma_close_5,ma_close_20,...,label_up,intraday_mean_ret_4h,intraday_std_ret_4h,intraday_n_up_4h,intraday_n_candles_4h,intraday_high_max,intraday_low_min,intraday_frac_up_4h,intraday_range_4h,intraday_last_ret_4h
0,2023-12-18,461.944824,462.933380,459.909019,460.975862,70375300,0.005625,0.005609,458.495721,448.292444,...,1,-0.001090,NaN,1.0,2.0,472.980011,470.799988,0.5,2.180023,-0.001090
1,2023-12-19,464.753906,464.832224,462.414688,462.492975,55761800,0.006081,0.006063,460.964941,449.389354,...,0,0.002973,0.001668,2.0,2.0,474.920013,472.450012,1.0,2.470001,0.001793
2,2023-12-20,458.313660,465.791362,457.883003,463.892568,102921000,-0.013857,-0.013954,460.898364,450.212508,...,1,-0.007190,0.012402,1.0,2.0,475.894989,467.820007,0.5,8.074982,-0.015960
3,2023-12-21,462.659332,462.933384,458.881307,461.318408,86667500,0.009482,0.009437,461.406531,451.167650,...,1,0.005020,0.000708,1.0,2.0,472.975006,468.839996,0.5,4.135010,0.005520
4,2023-12-22,463.589111,465.282375,461.680550,463.794642,67160400,0.002010,0.002008,462.252167,452.155632,...,1,0.000988,0.004317,1.0,2.0,475.380005,471.700012,0.5,3.679993,-0.002065


In [36]:
out_path = DATA_PROC / "daily_with_4h.parquet"
df_daily_merged.to_parquet(out_path, index=False)
out_path


WindowsPath('C:/Users/KDP only/Documents/ANN_Final_Project/spy-ann/data/processed/daily_with_4h.parquet')

## Notes for Report – 4-Hour Intraday Features

- We downloaded 4-hour SPY candles for the same period as the daily dataset.
- For each trading day, we aggregated 4h data into intraday summary features:
  - `intraday_n_candles_4h`: number of 4h candles that day.
  - `intraday_frac_up_4h`: fraction of 4h candles where Close > Open.
  - `intraday_mean_ret_4h`, `intraday_std_ret_4h`: mean and volatility of 4h returns.
  - `intraday_range_4h`: daily range from min Low to max High over 4h candles.
  - `intraday_last_ret_4h`: return of the last 4h candle of the day.
- These features are merged with the original daily technical feature table to form `daily_with_4h.parquet`.
- Days without sufficient intraday data (e.g., holidays) were dropped from the modeling dataset.
